<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Word Vectors

---

### Learning Objectives
- Describe word vectors and understand the shortcomings of bag-of-words methods.
- Describe word embeddings.
- Apply Word2Vec, GloVe, and BERT embedding techniques.

**We will start by importing what we need for Word2Vec, GloVe, and the transformer models.** (Downloading the pre-trained Word2Vec embeddings can take a while! We are using the [gensim.downloader](https://radimrehurek.com/gensim/auto_examples/howtos/run_downloader_api.html) module for this.)

In [1]:
# Install/upgrade Gensim & transformers
# !pip install gensim --upgrade
# !pip install transformers --upgrade

In [2]:
import gensim
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from transformers import pipeline

In [3]:
corpus = api.load('text8')

In [4]:
%%time
model = Word2Vec(corpus)

CPU times: user 2min 41s, sys: 1.47 s, total: 2min 43s
Wall time: 1min 2s


## Word Embeddings

### What is a vector?

There are lots of ways to think about a vector.

<img src="../images/vector.png" alt="drawing" width="450"/>

In **physics**, vectors are arrows.

<img src="../images/vector.jpg" alt="drawing" width="400"/>

In **computer science** and **statistics**, vectors are columns of values, like one numeric Series in a DataFrame.

#### It turns out that these are equivalent.

<img src="../images/vector_on_graph.png" alt="drawing" width="450"/>

[This video](https://www.youtube.com/watch?v=fNk_zzaMoSs) does an exceptional job explaining vectors.

### So... what is a word vector?

A word vector, simply, is a way for us to represent words with vectors.

<details><summary>How have we technically already done this?</summary>
    
- CountVectorizer and TFIDFVectorizer. By representing each word as a new column in our DataFrame, we have represented words with vectors.

![](../images/countvectorizer.jpeg)
</details>

To be more precise, we can think of each word as its own dimension or axis. In the example below, we have represented the horizontal axis with a vector for `cat` and the vertical axis with a vecvtor for `hat`.

<img src="../images/cat_hat.png" alt="drawing" width="400"/>

This is exactly what CountVectorization and TFIDFVectorization have done; we are now just representing it geometrically/visually! Each column in our DataFrame corresponds to a new axis.

This type of vectorization of words (turning each word into its own column) is known as "1-of-N encoding."

<img src="../images/one-hot-new.png" alt="drawing" width="400"/>

For example:
- the vector for the word `dog` would be [1, 0, 0, 0, 0].
- the vector for the word `cat` would be [0, 1, 0, 0, 0].
- the vector for the word `puppy` would be [0, 0, 1, 0, 0].
- the vector for the word `kitten` would be [0, 0, 0, 1, 0].
- the vector for the word `pug` would be [0, 0, 0, 0, 1].

All of the above vectors are independent of one another. Thinking purely about language and the way we use it, **should** dog and puppy be independent of one another? **Should** dog and pug be independent of one another?

<details><summary>What do you think?</summary>
    
- Probably not!
- Dog and puppy have similar meanings. (Really, only the age is different.)
- Dog and cat have similar meanings. (i.e. I know that "dog" and "cat" are more similar than "dog" and "book" or "cat" and "car.")
- Our current data science strategy for NLP (CountVectorization, TFIDFVectorization) is good in that it allows us to get computers to understand natural language in a way similar to how humans do... but our current strategy has its limitations!
</details>

Rather than creating a whole new dimension each time we encounter a new word and treating it as independent of all other words, can we instead come up with "new axes" that allow us to better understand meanings and relationships among words?
- YES.

**Word embedding** is a term used to describe representing words in mathematical space.
- One word embedding technique is CountVectorization.
- A more advanced word embedding technique is `Word2Vec`.

## Non-contextual Word Embeddings

### Word2Vec
- Word2Vec is an approach that takes in observations (sentences, tweets, books) and maps them into some other space using a neural network.

Going back to our previous example, you can "think" of a five-dimensional space. 
- The horizontal axis corresponds to `dog`.
- The vertical axis corresponds to `cat`.
- The axis extending out toward you corresponds to `puppy`.
- Given that we live in 3D space, we can't really visualize higher dimensions.

Instead of giving each word its own axis, the `Word2Vec` algorithm will take all of our words and map them to another set of axes that accounts for these relationships.

<img src="../images/word-vectors-new.png" alt="drawing" width="350"/>

### Why do we care?
The structure of language has a lot of valuable information in it! The way we organize our text/speech tells us a lot about what things mean.

By using machine learning to "learn" about the structure and content of language, our models can now organize concepts and learn the relationships among them.
- Above, we did not explicitly tell the computer what "dog" or "puppy" or "cat" or "kitten" actually mean. But by learning from the data, our model can quantify the relationship among these entities!

### How does Word2Vec work?

#### Basic Answer:
The idea is that we can use the position of words in sentences (i.e. see which words were commonly used together) to understand their relationships.
- If "dog" and "puppy" are used near one another a lot, then it suggests that there may be some sort of relationship between them.
- If "cat" and "dog" are used near similar words a lot (i.e. "pet"), then it suggests that there may be some sort of relationship between them.

#### More Advanced Answer:
There are two algorithms that use neural networks to learn these relationships: Continuous Bag-of-Words (CBOW) and Continuous Skip-grams.

![](../images/cbow.png)

**CBOW (BONUS)**

A continuous Bag-of-Words model is a two-layer neural network that:
- takes the surrounding "context words" as an input.
- generates the "focus word" as the output.

<img src="../images/word2vec-cbow.png" alt="drawing" width="400"/>

**Skip-Gram (BONUS)**

A Continuous Skip-gram model is a two-layer neural network that:
- takes the "focus word" as an input.
- generates the surrounding "context words" as the output.

<img src="../images/skipgram.png" alt="drawing" width="400"/>

([image source](https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/)).

### Neat application 1: Which of these is not like the other?

In [5]:
model.wv.doesnt_match(['dog', 'fish', 'cat', 'hamster', 'tiger']) 

/Users/noelle/opt/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'tiger'

In [6]:
model.wv.doesnt_match(['taco', 'burrito', 'quesadilla', 'hamburger'])

'hamburger'

In [7]:
model.wv.doesnt_match(['london', 'cannes', 'madrid', 'vienna'])

'cannes'

In [8]:
model.wv.doesnt_match(['angioplasty', 'appendectomy', 'cabg', 'bronchoscopy'])

'angioplasty'

Try your own and share the most mind-blowing one in a thread.

**Real-world application of this**: Suppose you're attempting to automatically detect spam emails or detect plagiarism based on words that don't belong.

### Neat application 2: What is most alike?

In [9]:
model.wv.most_similar('paris')

[('munich', 0.7387839555740356),
 ('venice', 0.73060542345047),
 ('vienna', 0.7199644446372986),
 ('milan', 0.7004169225692749),
 ('leipzig', 0.6951998472213745),
 ('louvre', 0.6894514560699463),
 ('naples', 0.6892068386077881),
 ('bologna', 0.6786153316497803),
 ('florence', 0.6685673594474792),
 ('villa', 0.6634528040885925)]

In [10]:
model.wv.most_similar('duodenum')

[('linea', 0.86106276512146),
 ('ureter', 0.8496915102005005),
 ('mesencephalon', 0.8437960743904114),
 ('diencephalon', 0.8417701721191406),
 ('efferent', 0.8406288027763367),
 ('capillaries', 0.8278489112854004),
 ('incision', 0.8263205289840698),
 ('ventricles', 0.8260178565979004),
 ('gustatory', 0.8235317468643188),
 ('pellets', 0.8220992684364319)]

In [11]:
model.wv.most_similar('mitochondria')

[('chloroplasts', 0.8996715545654297),
 ('organelles', 0.8826990127563477),
 ('membranes', 0.8724870681762695),
 ('flagella', 0.8629034757614136),
 ('proteins', 0.8608903884887695),
 ('eukaryotes', 0.8597884774208069),
 ('lipids', 0.8580181002616882),
 ('bacteria', 0.8460475206375122),
 ('tissues', 0.8441225290298462),
 ('archaea', 0.8411028981208801)]

In [12]:
model.wv.most_similar('emergency')

[('detention', 0.7014678120613098),
 ('monitoring', 0.6988388299942017),
 ('safety', 0.6974413394927979),
 ('surveillance', 0.6911908388137817),
 ('audit', 0.6491774916648865),
 ('contraception', 0.6433520913124084),
 ('prevention', 0.6353774070739746),
 ('relief', 0.6336306929588318),
 ('involuntary', 0.6309393048286438),
 ('humanitarian', 0.6259063482284546)]

**Real-world application of this**: Suppose you're building out a process to detect when people are tweeting about an emergency. They may not just use the word "emergency." Rather than manually creating a list of words people could use, you may want to learn from a much larger corpus of data than just your personal experience!

## Create Word2Vec word vectors from your own corpus! (BONUS)

### NOTE: This will usually take a *long* time!

In [13]:
# # Import Word2Vec
# from gensim.models.word2vec import Word2Vec

# # If you want to use gensim's data, import their downloader
# # and load it.
# import gensim.downloader as api
# corpus = api.load('text8')

# # If you have your own iterable corpus of cleaned data, you can 
# # read it in as corpus and pass that in.

# # Train a model! 
# model = Word2Vec(corpus,      # Corpus of data.
#                  size=100,    # How many dimensions do you want in your word vector?
#                  window=5,    # How many "context words" do you want?
#                  min_count=1, # Ignores words below this threshold.
#                  sg=0,        # SG = 1 uses SkipGram, SG = 0 uses CBOW (default).
#                  workers=4)   # Number of "worker threads" to use (parallelizes process).

# # Do what you'd like to do with your data!
# model.most_similar("car")

Check out the documentation for Gensim's implementation of [Word2Vec here](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec).

### GloVe

GloVe stands for Global Vectors for Word Representation. It is an unsupervised technique that maps words to vector representations where the distance between the vectors represents semantic similarities. This is done using a co-occurrence matrix which shows us how often pairs of words occur together.

In [14]:
model_glove = api.load("glove-wiki-gigaword-50")

### Neat application 1: Which of these is not like the other?

In [15]:
model_glove.doesnt_match(['dog', 'fish', 'cat', 'hampster', 'tiger']) 

'tiger'

### Neat application 2: What is most alike?

In [16]:
model_glove.most_similar("glass")

[('plastic', 0.7942505478858948),
 ('metal', 0.770871639251709),
 ('walls', 0.7700636386871338),
 ('marble', 0.7638524174690247),
 ('wood', 0.7624281048774719),
 ('ceramic', 0.7602593302726746),
 ('pieces', 0.7589111924171448),
 ('stained', 0.7528817057609558),
 ('tile', 0.748193621635437),
 ('furniture', 0.746385931968689)]

---
## Contextualized/Dynamic Word Embeddings

What are some shortcomings of `Word2Vec`? It takes into consideration the meaning of words based on context in the corpus, but what about words with different meanings?

How many meanings can you think of for the word "set"? This word [holds the record](https://www.guinnessworldrecords.com/world-records/english-word-with-the-most-meanings/) for the most number of meanings in the English language. Even a word like "apple" can take on vastly different meanings in today's age. `Word2Vec` assigns one vector for each word.

**Dynamic Word Embeddings** overcome this shortcoming by assigning an embedding to each word after looking at the sentence of the words. This means that the same words (e.g. "apple" in a sentence about fruit and "Apple" in a sentence about computers) can be represented by different vectors based on their contexts. One of the first popular models that did this was called **ELMo**. Another popular one is named **BERT**.

<img src="../images/bert.png" alt="drawing" width="200"/>

[BERT](https://github.com/google-research/bert) (Bidirectional Encoder Representations from Transformers) was created by Google in late 2018 and continues to outperform other language representation models. It combined ELMo and several other transformers and is fully bidirectional allowing words to have different vectors based on the context of the word.

BERT is an example of a Transformer model. The following is from [Wikipedia](https://en.wikipedia.org/wiki/Transformer_(machine_learning_model)):

> Like recurrent neural networks (RNNs), Transformers are designed to handle sequential data, such as natural language, for tasks such as translation and text summarization. However, unlike RNNs, Transformers do not require that the sequential data be processed in order. For example, if the input data is a natural language sentence, the Transformer does not need to process the beginning of it before the end. Due to this feature, the Transformer allows for much more parallelization than RNNs and therefore reduced training times.  
Since their introduction, Transformers have become the model of choice for tackling many problems in NLP, replacing older recurrent neural network models such as the long short-term memory (LSTM).

We will use Hugging Face's [transformers](https://github.com/huggingface/transformers) for this section.

### Neat application 1: Fill in the blank
We will use the BERT model here!

In [17]:
unmasker = pipeline('fill-mask', model='bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
unmasker('I want to go for a [MASK] in the park')

[{'sequence': '[CLS] i want to go for a walk in the park [SEP]',
  'score': 0.9612457156181335,
  'token': 3328,
  'token_str': 'walk'},
 {'sequence': '[CLS] i want to go for a run in the park [SEP]',
  'score': 0.008957462385296822,
  'token': 2448,
  'token_str': 'run'},
 {'sequence': '[CLS] i want to go for a ride in the park [SEP]',
  'score': 0.00624713534489274,
  'token': 4536,
  'token_str': 'ride'},
 {'sequence': '[CLS] i want to go for a stroll in the park [SEP]',
  'score': 0.0060236454010009766,
  'token': 27244,
  'token_str': 'stroll'},
 {'sequence': '[CLS] i want to go for a swim in the park [SEP]',
  'score': 0.0045113470405340195,
  'token': 9880,
  'token_str': 'swim'}]

### Neat application 2: Sentiment Analysis
This was trained on [sst2](https://www.tensorflow.org/datasets/catalog/glue).

In [19]:
sent = pipeline('sentiment-analysis')

In [20]:
sent('This was the worst movie ever!')

[{'label': 'NEGATIVE', 'score': 0.9997475147247314}]

In [21]:
sent('I love this so much')

[{'label': 'POSITIVE', 'score': 0.9998810291290283}]

### Neat application 3: Question Answering
This was trained on [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/).

In [22]:
question = pipeline('question-answering')

In [23]:
ga = 'General Assembly is a pioneer in education and career transformation, specializing in today\'s most in-demand coding, business, data, and design skills. With 30+ campuses around the world, we provide award-winning, dynamic training to a global community of professionals pursuing careers they love. Named one of Fast Company’s most innovative education companies, GA offers full- and part-time courses for career climbers both on campus and online. Through our corporate training programs, we also help companies compete for the future by sourcing, assessing, and growing their talent. All of these offerings are developed and led by industry experts.'
print(ga)

General Assembly is a pioneer in education and career transformation, specializing in today's most in-demand coding, business, data, and design skills. With 30+ campuses around the world, we provide award-winning, dynamic training to a global community of professionals pursuing careers they love. Named one of Fast Company’s most innovative education companies, GA offers full- and part-time courses for career climbers both on campus and online. Through our corporate training programs, we also help companies compete for the future by sourcing, assessing, and growing their talent. All of these offerings are developed and led by industry experts.


In [24]:
question(context = ga, question = 'Where is General Assembly located?')

/Users/noelle/opt/anaconda3/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1374: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


{'score': 0.5068284869194031,
 'start': 157,
 'end': 187,
 'answer': '30+ campuses around the world,'}

In [25]:
question(context = ga, question = 'What can I learn at General Assembly?')

/Users/noelle/opt/anaconda3/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1374: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


{'score': 0.19035623967647552,
 'start': 99,
 'end': 151,
 'answer': 'in-demand coding, business, data, and design skills.'}

### Neat application 4: Summarization
By default, this uses a [Bart](https://medium.com/analytics-vidhya/assesing-barts-syntactic-abilities-and-bert-s-part-1-cbf0983f6ea4) model that was trained on CNN/Daily Mail data.

In [26]:
summarizer = pipeline('summarization')

In [27]:
# https://www.upi.com/Odd_News/2020/10/22/Bear-opens-car-door-climbs-inside-in-Tennessee/9821603398162/
news = """
An Indiana family visiting Tennessee captured video of a black bear wandering up to their unoccupied car, opening a door and climbing inside.
The Franczak family said they traveled from Crown Point, Ind., to Sevierville, Tenn., to celebrate a grandmother's birthday. "One of our bucket list things was to see a bear," father Brian Franczak told WBBM-TV.
The family said they were shocked, however, when a bear came walking up the driveway of their vacation home and headed for their SUV.
"I just screamed, 'Oh my God! The bear is here! The bear is in the driveway,'" mom Carly Franczak said.
The family captured video as the bear opened a back door of the vehicle and climbed inside.
"I was at go-carts racing and my grandpa got a call about that there's a bear in their car," daughter Olivia Franczak said, "and we couldn't believe it at first. We thought my uncle got dressed up as a bear and went into the car."
The Tennessee Wildlife Resources Agency recommends residents and visitors keep vehicle doors locked at all times and make sure food and trash are secured where the animals can't reach.
"""

In [28]:
summarizer(news)

[{'summary_text': " The Franczak family was visiting Tennessee to celebrate a grandmother's birthday . The family said they were shocked when a bear came walking up to their car . The bear climbed into the vehicle and climbed inside . The Tennessee Wildlife Resources Agency recommends residents and visitors keep vehicle doors locked ."}]

### Neat application 5: Text Generation
Using [GPT-2](https://en.wikipedia.org/wiki/OpenAI#GPT-2)!

In [29]:
text_generator = pipeline("text-generation")

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
text_generator("I would like to")

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[{'generated_text': 'I would like to thank the following sponsors (the sponsors who helped shape the design of "The Great Beyond):\n\nSleeper Energy Sports\n\nBeverly C. Anderson\n\nPillars for Life Sports\n\nLeverage'}]

## (BONUS) Applying this to your data

Want to use a pre-trained model on your own text data? Due to hardware and time limitations, we will not do this in class, but below are several tutorials that can walk you through this. Warning: these models take a lot of time/memory - you may need a GPU for this! ([Google Colab offers free use of a GPU!](https://www.tutorialspoint.com/google_colab/google_colab_using_free_gpu.htm))

- [Example of BERT in Keras](https://colab.research.google.com/drive/1934Mm2cwSSfT5bvi78-AExAl-hSfxCbq#scrollTo=gsscu_BluPLE)
- [BERT tutorial](https://towardsml.com/2019/09/17/bert-explained-a-complete-guide-with-theory-and-tutorial/)
- [Predicting movie review sentiment with BERT](https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb#scrollTo=dCpvgG0vwXAZ)
- [Classification with BERT in PyTorch](https://colab.research.google.com/drive/1ywsvwO6thOVOrfagjjfuxEf6xVRxbUNO)
- [Classification with GloVe embeddings](https://medium.com/analytics-vidhya/text-classification-using-word-embeddings-and-deep-learning-in-python-classifying-tweets-from-6fe644fcfc81)
- [Using pre-trained word embeddings in Keras](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html)